In [1]:
from google.colab import files

uploaded = files.upload()

ModuleNotFoundError: No module named 'google'

In [2]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import (confusion_matrix, accuracy_score)

In [3]:
data = pd.read_csv("Music_Genre.csv")

In [5]:
data.shape

(17996, 17)

In [6]:
data.head()

,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Class
0,Bruno Mars,That's What I Like (feat. Gucci Mane),60.0,0.854,0.564,1.0,-4.964,1,0.0485,0.017100,NaN,0.0849,0.8990,134.071,234596.0,4,5
1,Boston,Hitch a Ride,54.0,0.382,0.814,3.0,-7.230,1,0.0406,0.001100,0.004010,0.1010,0.5690,116.454,251733.0,4,10
2,The Raincoats,No Side to Fall In,35.0,0.434,0.614,6.0,-8.334,1,0.0525,0.486000,0.000196,0.3940,0.7870,147.681,109667.0,4,6
3,Deno,Lingo (feat. J.I & Chunkz),66.0,0.853,0.597,10.0,-6.528,0,0.0555,0.021200,NaN,0.1220,0.5690,107.033,173968.0,4,5
4,Red Hot Chili Peppers,Nobody Weird Like Me - Remastered,53.0,0.167,0.975,2.0,-4.279,1,0.2160,0.000169,0.016100,0.1720,0.0918,199.060,229960.0,4,10


In [7]:
data["duration"] = data["duration_in min/ms"]
data["timesignature"] = data["time_signature"]

In [8]:
# 7=Instrumental Metal=8; Pop=9
data = data.loc[data['Class'].isin([7, 8, 9])]

In [9]:
data['Class'] = data['Class'].replace({7: 0, 8: 1, 9: 2})

In [10]:
data = data.dropna(subset=['Popularity', 'danceability', 'loudness', 'tempo', 'duration', 'timesignature'])

In [11]:
columns_to_round = ['Popularity', 'danceability', 'loudness', 'tempo', 'duration', 'timesignature']

data[columns_to_round] = data[columns_to_round].round()

In [12]:
X = data.drop('Class', axis=1)
y = data['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
X_train["Class"] = y_train
X_test["Class"] = y_test

In [15]:
# 0=Instrumental 1=Metal 2=Pop
model = smf.mnlogit('Class ~ Popularity + danceability + loudness + tempo + duration + C(timesignature)', data=X_train).fit()

print(model.summary())

Optimization terminated successfully.
         Current function value: 0.371008
         Iterations 21
                          MNLogit Regression Results                          
Dep. Variable:                  Class   No. Observations:                 3864
Model:                        MNLogit   Df Residuals:                     3846
Method:                           MLE   Df Model:                           16
Date:                Fri, 12 Jan 2024   Pseudo R-squ.:                  0.6108
Time:                        10:54:00   Log-Likelihood:                -1433.6
converged:                       True   LL-Null:                       -3683.2
Covariance Type:            nonrobust   LLR p-value:                     0.000
              Class=1       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 5.1718      2.867      1.804      0.071      -0.447      10

In [16]:
np.exp(model.params)

,0,1
Intercept,176.238380,251.094203
C(timesignature)[T.3],0.075633,0.194870
C(timesignature)[T.4],1.050925,2.173538
C(timesignature)[T.5],0.112678,0.451983
Popularity,1.007942,1.024695
danceability,0.052838,0.526835
loudness,2.610487,1.972840
tempo,1.009598,1.005252
duration,1.000143,1.000125


In [17]:
newpred = pd.DataFrame({"Popularity": [0], "timesignature": [3], "danceability": [200], "loudness": 0, "duration": 0, "tempo": [0]})
model.predict(newpred)

,0,1,2
0,1.0,5.171974e-255,1.057881e-54
